In [4]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/synthetic_logs.csv")
df.head()


,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [5]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [6]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN


# Generate embeddings for log messages
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['log_message'].tolist())


embeddings[:2]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[-1.02939688e-01,  3.35459262e-02, -2.20260695e-02,
         1.55106047e-03, -9.86916292e-03, -1.78956240e-01,
        -6.34410083e-02, -6.01761751e-02,  2.81108599e-02,
         5.99619560e-02, -1.72618553e-02,  1.43369031e-03,
        -1.49560034e-01,  3.15284147e-03, -5.66031411e-02,
         2.71685403e-02, -1.49890585e-02, -3.54037657e-02,
        -3.62936892e-02, -1.45411231e-02, -5.61499177e-03,
         8.75538662e-02,  4.55120504e-02,  2.50963811e-02,
         1.00187277e-02,  1.24266976e-02, -1.39923587e-01,
         7.68696368e-02,  3.14095207e-02, -4.15258622e-03,
         4.36902940e-02,  1.71250608e-02, -8.00950751e-02,
         5.74006028e-02,  1.89092010e-02,  8.55261311e-02,
         3.96399386e-02, -1.34371847e-01, -1.44363963e-03,
         3.06707947e-03,  1.76854074e-01,  4.44880035e-03,
        -1.69275012e-02,  2.24266537e-02, -4.35049571e-02,
         6.09025033e-03, -9.98168718e-03, -6.23972267e-02,
         1.07372599e-02, -6.04897691e-03, -7.14660436e-0

In [8]:
# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

# Add cluster labels to the DataFrame
df['cluster'] = clusters
df.head(100)

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0
...,...,...,...,...,...,...
95,10/19/2025 17:15,ThirdPartyAPI,Detection of admin privilege misuse by user 1449,Security Alert,bert,31
96,12/7/2025 10:38,ModernCRM,Disk cleanup completed successfully.,System Notification,regex,32
97,11/1/2025 12:05,AnalyticsEngine,"Server 40 is under potential security threat, ...",Security Alert,bert,33
98,12/7/2025 8:23,ModernCRM,Data replication task failed for shard 17,Error,bert,3


In [9]:
df[df.cluster==1]

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,bert,1
450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,bert,1
477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,bert,1
570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,bert,1
678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,bert,1


In [10]:


# Count the number of records in each cluster
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts > 10].index


# Print 5 log messages from clusters with more than 10 records
for cluster in large_clusters:
  print(f"Cluster {cluster}:")
  print(df[df['cluster'] == cluster]['log_message'].head(5).to_string(index=False))
  print()

Cluster 0:
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

Cluster 5:
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

Cluster 11:
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logged in.

Cluster 13:
Backup started at 2025-05-14 07:06:55.
Backup started at 2025-02-15 20:00:19.
  Backup ended at 2025-08-08 13:06:23.
Backup started at 2025-11-14 08:27:43.
Backup started at 2025-12-09 10:19:11.

Cluster 7:
Multiple bad login attempts detected on user 85...
Multiple login failures occurred on user 9052 a...
  User 

In [11]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None


In [12]:
classify_with_regex("disk cleanup completed successfully.")

'System Notification'

In [13]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()]  # 500 rows classified by regex

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [14]:
df.shape

(2410, 7)

In [15]:
df_non_regex = df[df.regex_label.isnull()].copy()
df_non_regex.shape

(1910, 7)

In [16]:
print(df_non_regex.target_label.value_counts()[df_non_regex['target_label'].value_counts() <= 5].index.tolist())

# if we have enough samples we can generate embedding in BERT
# if  not enough then we go to LLM

['Workflow Error', 'Deprecation Warning']


In [17]:
df_non_legacy = df_non_regex[df_non_regex.source!='LegacyCRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [18]:

filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())


filtered_embeddings[:2]

array([[-1.02939688e-01,  3.35459262e-02, -2.20260695e-02,
         1.55106047e-03, -9.86916292e-03, -1.78956240e-01,
        -6.34410083e-02, -6.01761751e-02,  2.81108599e-02,
         5.99619560e-02, -1.72618553e-02,  1.43369031e-03,
        -1.49560034e-01,  3.15284147e-03, -5.66031411e-02,
         2.71685403e-02, -1.49890585e-02, -3.54037657e-02,
        -3.62936892e-02, -1.45411231e-02, -5.61499177e-03,
         8.75538662e-02,  4.55120504e-02,  2.50963811e-02,
         1.00187277e-02,  1.24266976e-02, -1.39923587e-01,
         7.68696368e-02,  3.14095207e-02, -4.15258622e-03,
         4.36902940e-02,  1.71250608e-02, -8.00950751e-02,
         5.74006028e-02,  1.89092010e-02,  8.55261311e-02,
         3.96399386e-02, -1.34371847e-01, -1.44363963e-03,
         3.06707947e-03,  1.76854074e-01,  4.44880035e-03,
        -1.69275012e-02,  2.24266537e-02, -4.35049571e-02,
         6.09025033e-03, -9.98168718e-03, -6.23972267e-02,
         1.07372599e-02, -6.04897691e-03, -7.14660436e-0

In [19]:
X = filtered_embeddings
y = df_non_legacy['target_label']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [22]:
import joblib

joblib.dump(clf, '/content/drive/My Drive/Colab Notebooks/log_classifier.joblib')



['/content/drive/My Drive/Colab Notebooks/log_classifier.joblib']